# 位置が近すぎるラーメン屋を集約して点数の高い方を残す

In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv(os.path.join(os.pardir, 'data', 'store.csv'))
df.head()

,name,score,station,tel,address,latitude,longitude,open_time,closed_days,sheets,url,icon_url
0,銀座 八五,3.90,東銀座駅,不明の為情報お待ちしております,東京都中央区銀座3-14-2 第一はなぶさビル 1F,35.670547,139.770150,11:00～ 当日の並び順でのご案内※当日分のスープがなくなり次第受付を終了いたします。12...,月曜日、火曜日（不定休）※定休日の最新情報は、twitterをご覧ください。,6席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13228602/,https://tblg.k-img.com/resize/640x640c/restaur...
1,むぎとオリーブ 銀座本店,3.78,東銀座駅,03-3571-2123,東京都中央区銀座6-12-12 銀座ステラビル 1F,35.669003,139.764360,[月・火・木・金・土・日]11:00～15:30(L.O.)17:30～21:30(L.O.),水曜日,15席 （カウンター15席）,https://tabelog.com/tokyo/A1301/A130101/13164932/,https://tblg.k-img.com/resize/640x640c/restaur...
2,麺屋ひょっとこ 交通会館店,3.78,有楽町駅,03-3211-6002,東京都千代田区有楽町2-10-1 東京交通会館 B1F,35.675160,139.764343,[月～金]11:00～20:00[土・祝]11:00～19:00,日曜日,7席 （カウンター7席）,https://tabelog.com/tokyo/A1301/A130102/13046637/,https://tblg.k-img.com/resize/640x640c/restaur...
3,銀座 朧月（おぼろづき）,3.78,銀座駅,03-6280-6166,東京都中央区銀座6-3-5 第一高橋ビル 1F,35.671578,139.761383,【月・火・水・日祝】11:00～16:00 17:00～22:00【木・金・土】11:00〜...,無休,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13118141/,https://tblg.k-img.com/resize/640x640c/restaur...
4,麺処 銀笹（ぎんざさ）,3.78,築地市場駅,03-3543-0280,東京都中央区銀座8-15-2 藤ビル 1F,35.666225,139.763421,[月～金]11:30～15:0017:30～22:00(スープがなくなり次第営業終了)[土]...,日曜日、祝日,18席 （４人掛けテーブル４卓、２人掛けテーブル１卓）,https://tabelog.com/tokyo/A1301/A130103/13119563/,https://tblg.k-img.com/resize/640x640c/restaur...


In [3]:
df['x'] = 1
df2 = pd.merge(df, df, on='x', how='inner', suffixes=['', '_'])[
    ['url', 'score', 'latitude', 'longitude', 
     'url_', 'score_', 'latitude_', 'longitude_']]
df2['distance'] = (df2['latitude'] - df2['latitude_']) ** 2 + (df2['longitude'] - df2['longitude_'])  ** 2
del df['x']
df2.head()

,url,score,latitude,longitude,url_,score_,latitude_,longitude_,distance
0,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670547,139.77015,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.90,35.670547,139.770150,0.000000
1,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670547,139.77015,https://tabelog.com/tokyo/A1301/A130101/13164932/,3.78,35.669003,139.764360,0.000036
2,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670547,139.77015,https://tabelog.com/tokyo/A1301/A130102/13046637/,3.78,35.675160,139.764343,0.000055
3,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670547,139.77015,https://tabelog.com/tokyo/A1301/A130101/13118141/,3.78,35.671578,139.761383,0.000078
4,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670547,139.77015,https://tabelog.com/tokyo/A1301/A130103/13119563/,3.78,35.666225,139.763421,0.000064


In [4]:
# 距離が近い店舗の中で最大のスコアを持つ店舗を抽出
for i, threshold in enumerate([0.00001, 0.0001]):
    tmp = df2[df2['distance'] < threshold].groupby(['url', 'score'], as_index=False)[['score_']].max()
    tmp = tmp[tmp['score'] == tmp['score_']]
    df[f'summarize_{i}'] = (df['url'].isin(set(tmp['url']))).astype(int)
df.head()

,name,score,station,tel,address,latitude,longitude,open_time,closed_days,sheets,url,icon_url,summarize_0,summarize_1
0,銀座 八五,3.90,東銀座駅,不明の為情報お待ちしております,東京都中央区銀座3-14-2 第一はなぶさビル 1F,35.670547,139.770150,11:00～ 当日の並び順でのご案内※当日分のスープがなくなり次第受付を終了いたします。12...,月曜日、火曜日（不定休）※定休日の最新情報は、twitterをご覧ください。,6席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13228602/,https://tblg.k-img.com/resize/640x640c/restaur...,1,1
1,むぎとオリーブ 銀座本店,3.78,東銀座駅,03-3571-2123,東京都中央区銀座6-12-12 銀座ステラビル 1F,35.669003,139.764360,[月・火・木・金・土・日]11:00～15:30(L.O.)17:30～21:30(L.O.),水曜日,15席 （カウンター15席）,https://tabelog.com/tokyo/A1301/A130101/13164932/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
2,麺屋ひょっとこ 交通会館店,3.78,有楽町駅,03-3211-6002,東京都千代田区有楽町2-10-1 東京交通会館 B1F,35.675160,139.764343,[月～金]11:00～20:00[土・祝]11:00～19:00,日曜日,7席 （カウンター7席）,https://tabelog.com/tokyo/A1301/A130102/13046637/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
3,銀座 朧月（おぼろづき）,3.78,銀座駅,03-6280-6166,東京都中央区銀座6-3-5 第一高橋ビル 1F,35.671578,139.761383,【月・火・水・日祝】11:00～16:00 17:00～22:00【木・金・土】11:00〜...,無休,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13118141/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
4,麺処 銀笹（ぎんざさ）,3.78,築地市場駅,03-3543-0280,東京都中央区銀座8-15-2 藤ビル 1F,35.666225,139.763421,[月～金]11:30～15:0017:30～22:00(スープがなくなり次第営業終了)[土]...,日曜日、祝日,18席 （４人掛けテーブル４卓、２人掛けテーブル１卓）,https://tabelog.com/tokyo/A1301/A130103/13119563/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0


In [5]:
df.to_csv(os.path.join(os.pardir, 'data', 'store_summarize.csv'))